# Assignment 1

In [2]:
import re
import helper
import numpy as np
import pandas as pd

ODI_DATASET_PATH = './data/ODI-2022.csv'

In [3]:
# # load csv file to pandas dataframe to retrieve column names
# df_temp= pd.read_csv("./data/ODI-2022.csv", delimiter=";")
# df_temp

In [4]:
# Initialize 2 dimensional array
dataset = helper.get_data(ODI_DATASET_PATH)
# Initialize columns
columns = list(zip(*dataset))

# print("What programme are you in? = {}".format(columns[1]))
# print("Have you taken a course on machine learning? = {}".format(columns[2]))
# print("Have you taken a course on information retrieval?; = {}".format(columns[3]))
# # mu is yes, sigma is no
# print("Have you taken a course on statistics? = {}".format(columns[4]))
# print("Have you taken a course on databases? = {}".format(columns[5]))
# print("What is your gender? = {}".format(columns[6]))
# print("Chocolate makes you..... = {}".format(columns[7]))
# print("When is your birthday (date)? = {}".format(columns[8]))
# print("Number of neighbors sitting around you? = {}".format(columns[9]))
# print("Did you stand up? = {}".format(columns[10]))
# print("What is your stress level (0-100)? = {}".format(columns[11]))
# print("You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then? = {}".format(columns[12]))
# print("Give a random number = {}".format(columns[13]))
# print("Time you went to be Yesterday = {}".format(columns[14]))
# print("What makes a good day for you (1)?; = {}".format(columns[15]))
# print("What makes a good day for you (2)? = {}".format(columns[16]))



/Users/rvandelaar/Desktop/School/Master AI/DMT/dmt-assignment-1/assignment_1/helper.py:5: ConversionWarning: Some errors were detected !
    Line #150 (got 13 columns instead of 17)
    Line #207 (got 18 columns instead of 17)
  array = np.genfromtxt(file_name, dtype=datatype, delimiter=";", invalid_raise=False, skip_header=True)


In [24]:
# load csv file to pandas dataframe to retrieve column names
df_temp= pd.read_csv("./data/ODI-2022.csv", delimiter=";")
column_names = df_temp.columns

# convert dataset to pandas dataframe using column names from df_temp
df = pd.DataFrame(dataset)
df.columns=column_names

df_temp = df_temp.drop(labels=148, axis=0)
df_temp = df_temp.drop(labels=205, axis=0)

## clean all column values
# remove unimportant columns
df = df.drop("Tijdstempel", axis=1) # remove Tijdstempel column because it has no meaning
df = df.drop("Did you stand up?", axis=1) # remove stand up column because it has no meaning
df = df.drop("Number of neighbors sitting around you?", axis=1) # remove neighbours column because it has no meaning
df = df.drop("Give a random number", axis=1) # remove random number column because it has no meaning

# convert column values to boolean for yes/no questions
df['Have you taken a course on machine learning?'] = df['Have you taken a course on machine learning?'].replace(['yes','no','unknown'],[True,False,False])
df['Have you taken a course on information retrieval?'] = df['Have you taken a course on information retrieval?'].replace([1,0,-1],[True,False, False])
df['Have you taken a course on statistics?'] = df['Have you taken a course on statistics?'].replace(['mu','sigma', 'unknown'],[True,False,False])
df['Have you taken a course on databases?'] = df['Have you taken a course on databases?'].replace(['ja','nee', 'unknown'],[True,False,False])

# only keep years in birthday column
def get_year(date):
    year = re.search('[0-9]{4}', date)
    symbols = {'/', '.', '-'}
    if year == None:
        if len(date) > 4 and any(sym in date for sym in symbols):
            year = "19" + date[-2:]
            if year.isdecimal():
                year = int(year)
                if 1930 > year:
                    return np.nan
                elif year > 2005:
                    return np.nan
                else:
                    return year
            else:
                return np.nan
        else:
            return np.nan
    else:
        year = int(year.group(0))
        if 1930 > year:
            return np.nan
        elif year > 2005:
            return np.nan
        else:
            return year

# iterate all rows of birthday column and set all birthdays to year
for index, row in df.iterrows():
    birthday = (row["When is your birthday (date)?"])
    new_birthday = get_year(birthday)
    df['When is your birthday (date)?'] = df['When is your birthday (date)?'].replace([birthday],new_birthday)

# clean stress level column --> set all values < 0 to 0 and > 100 to 100
df = df.astype({'What is your stress level (0-100)?': int}, errors='raise')
df['What is your stress level (0-100)?'].values[df['What is your stress level (0-100)?'].values > 100] = 100
df['What is your stress level (0-100)?'].values[df['What is your stress level (0-100)?'].values < 0] = 0

# clean competition column --> set all values < 0 to 0 and > 100 to 100
df.rename(columns = {'You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then? ':'competition'}, inplace = True)
df = df.astype({'competition': int}, errors='raise')
df['competition'].values[df['competition'].values > 100] = 100
df['competition'].values[df['competition'].values < 0] = 0

# clean bedtime column
df.rename(columns={'Time you went to be Yesterday': 'bedtime'}, inplace=True)

# for new, old in zip(list(df_temp['Time you went to be Yesterday']), list(df['bedtime'])):
#     df['bedtime'] = df['bedtime'].replace([old], new)

# for new, old in zip(list(df_temp['Time you went to be Yesterday']), list(df['bedtime'])):
#     print(new, old)

def clean_time(old):
    old = old.lower()
    if re.match('^[0-9]{2}\:[0-9]{2}$', old):
        return old
    elif re.match('^[0-9]{2}$', old):
        keep = {'9', '10', '11', '12', '21', '22', '23', '24'}
        if old in keep:
            new = old + ':' + '00'
            return new
        else:
            return np.nan
    elif len(old) == 1:
        return np.nan
    elif re.match('^[0-9]{4}$', old):
        return old[0:2] + ':' + old[2:4]
    elif old[-2:] == 'am':
        print(f'Match!: {old}.')
        digits = re.sub('[^0-9]', '', old)
        if len(digits) == 4:
            return old[0:2] + ':' + old[2:4]
        elif len(digits) == 3:
            return '0' + ':' + old[1:4]
        elif len(digits) == 2:
            print(digits)
        else:
            return '0' + digits + ':00'
    else:
        print("Fail")
    # if ':' in old:
    #     old = old.replace(':', '')
    #     if old.isdigit():
    #         return old
    #     else:
    #         print(old)
    # else:
    #     print(old)

for index, row in df.iterrows():
    old_time = (row['bedtime'])
    new_time = clean_time(old_time)
    print(new_time)
    print()
    # df['bedtime'] = df['bedtime'].replace([old_time], new_time)

display(df)


# What kinds of attributes are there
## Boolean, object, integer, float
# print(df['What is your gender?'].unique())
# print(df.dtypes)
# display(df)


# Ranges of values
# Distribution of values,
# relationships between attributes
# and so on.

23:00

nan

nan

14:00

07:20

21:30

00:35

23:00

23:00

Fail
None

Fail
None

00:00

Fail
None

00:00

12:00

Fail
None

Fail
None

12:55

12:00

12:00

20:00

Fail
None

03:00

Fail
None

Match!: 2.30am.
0:.30

Fail
None

23:32

nan

12:00

Fail
None

12:30

nan

23:00

nan

nan

Fail
None

nan

Fail
None

Fail
None

09:00

Fail
None

Fail
None

Fail
None

11:00

23:30

nan

23:00

02:00

Fail
None

Fail
None

nan

01:00

Fail
None

23:45

24:00

Fail
None

Fail
None

Fail
None

00:34

13:15

nan

Fail
None

Fail
None

23:00

22:30

15:30

12:00

Fail
None

Fail
None

Match!: 2 am.
02:00

23:00

Match!: 0530 am.
05:30

12:00

13:12

Fail
None

Fail
None

Fail
None

12:00

12:00

Fail
None

12:00

Fail
None

10:45

Fail
None

00:00

10:00

22:00

24:00

12:00

12:45

Match!: 6 am.
06:00

23:00

Fail
None

12:00

Fail
None

nan

Match!: 9.00 am.
0:.00

Fail
None

10:00

00:45

Fail
None

nan

23:00

Fail
None

Fail
None

Fail
None

12:00

00:55

23:30

12:00

Fail
None

01:00

23:30


,What programme are you in?,Have you taken a course on machine learning?,Have you taken a course on information retrieval?,Have you taken a course on statistics?,Have you taken a course on databases?,What is your gender?,Chocolate makes you.....,When is your birthday (date)?,What is your stress level (0-100)?,competition,bedtime,What makes a good day for you (1)?,What makes a good day for you (2)?
0,artificial intelligence,True,False,True,False,female,neither,1998.0,0,5,23:00,coffee,sun
1,big data,True,False,True,False,male,fat,2000.0,100,1,32,7777,8964
2,Computational Science,True,True,True,True,male,fat,1998.0,0,100,x,0,0
3,Bioinformatics and Systems Biology,True,False,False,False,male,neither,1997.0,20,100,14:00,good food,no stress
4,AI,True,True,True,True,male,fat,1997.0,2,100,07:20,Sunshine,Good Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Ai,True,True,False,False,not willing to answer,fat,1956.0,30,99,5,Yes,No
298,E,True,True,True,True,male,slim,NaN,9,5,9,9,9
299,MSc Computational Science,False,False,True,False,male,I have no idea what you are talking about,1995.0,100,1,555,Good food,Sleep more than 6 hours
300,Artificial Intelligence,True,False,True,False,not willing to answer,neither,NaN,70,87,0,nice weather,good mood


In [6]:
# How many records are there
n_records = df.shape[0]
print(f"There are {n_records} records in the file.")

# How many attributes?
n_attributes = len(df.columns)
print(f"There are {n_attributes} attributes in the file.")

# Make various plots of the data. Is there something interesting worth reporting? Re-
# port the figures, discuss what is in them. What meaning do those bars, lines, dots, etc.
# convey? Please select essential and interesting plots for discussion, as you have limited
# space for reporting your findings (see details in a later section).